# Metrics

In this notebook we test any remaining metrics we need need after training

## Results

### ResNet

1. Total Parameters: 466906
2. MACs: 34879808
3. Energy FP32: 0.160 mJ
4. Energy FP16: 0.038 mJ
5. Energy INT8: 0.007 mJ
6. Energy reduction FP32: 4.2×
7. Energy reduction INT8: 23.0×

### SqueezeNet

1. Total Parameters: 734986
2. MACs: 26637312
3. Energy FP32: 0.123 mJ
4. Energy FP16: 0.029 mJ
5. Energy INT8: 0.005 mJ
6. Energy reduction FP32: 4.2×
7. Energy reduction INT8: 23.0×

### AlexNet

1. Total Parameters: 1048330
2. MACs: 33598720
3. Energy FP32: 0.155 mJ
4. Energy FP16: 0.037 mJ
5. Energy INT8: 0.007 mJ
6. Energy reduction FP32: 4.2×
7. Energy reduction INT8: 23.0×

## Energy vs Precision

In [4]:
import torch

import helper
from squeezenet_model import SqueezeNetCIFAR10, SqueezeNetCIFAR10_QAT
from alexnet_model import AlexNetCIFAR10, AlexNetCIFAR10_QAT
from resnet32_model import ResNet, ResNetQAT

In [25]:
# mname = "squeezenet"
mname = "alexnet"
# mname = "resnet"

if mname == "squeezenet":
    get_model = SqueezeNetCIFAR10
    get_model_qat = SqueezeNetCIFAR10_QAT
elif mname == "alexnet":
    get_model = AlexNetCIFAR10
    get_model_qat = AlexNetCIFAR10_QAT
elif mname == "resnet":
    get_model = ResNet
    get_model_qat = ResNetQAT

In [26]:
def estimate_energy(macs, precision="fp32"):
    energy_per_mac = {
        "fp32": 4.6e-12,
        "fp16": 1.1e-12,
        "int8": 0.2e-12
    }
    return macs * energy_per_mac[precision]  # Joules

def compute_sparsity(model):
    total = 0
    zeros = 0
    for m in model.modules():
        if isinstance(m, (torch.nn.Conv2d, torch.nn.Linear)):
            w = m.weight.detach()
            total += w.numel()
            zeros += (w == 0).sum().item()
    return zeros / total

In [33]:
from fvcore.nn import FlopCountAnalysis

def get_metrics(mname="squeezenet"):
    print(f"{mname} Metrics")

    model = get_model()
    model.load_model(f"../pth/{mname}_fp32.pth", device='cpu')
    x = torch.randn(1, 3, 32, 32)

    # total parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total Parameters: {total_params}")

    # total MACs
    flops = FlopCountAnalysis(model, x)
    macs = flops.total() // 2  # FLOPs = 2 × MACs
    print(f"MACs: {macs}")

    # energy vs precision
    energy_fp32 = estimate_energy(macs, "fp32")
    energy_fp16 = estimate_energy(macs, "fp16")
    energy_int8 = estimate_energy(macs, "int8")
    print(f"Energy FP32: {energy_fp32*1e3:.3f} mJ")
    print(f"Energy FP16: {energy_fp16*1e3:.3f} mJ")
    print(f"Energy INT8: {energy_int8*1e3:.3f} mJ")
    print(f"Energy reduction FP32: {energy_fp32 / energy_fp16:.1f}×")
    print(f"Energy reduction INT8: {energy_fp32 / energy_int8:.1f}×")

    #
    fnames = [f"{mname}_10", f"{mname}_30", f"{mname}_50", f"{mname}_70"]

    for fn in fnames:
        model_pruned = get_model()
        model_pruned.load_model(f"../pth/{fn}.pth", device='cpu')
        sparsity = compute_sparsity(model_pruned)
        eff_macs = macs * (1 - sparsity)
        energy = estimate_energy(eff_macs,"fp32")
        print(f"Sparsity: {sparsity*100:.1f}%")
        print(f"Energy: {energy*1e3:.3f}")

get_metrics(mname)

Unsupported operator aten::max_pool2d encountered 3 time(s)


alexnet Metrics
Model loaded from ../pth/alexnet_fp32.pth
Total Parameters: 1048330
MACs: 33598720
Energy FP32: 0.155 mJ
Energy FP16: 0.037 mJ
Energy INT8: 0.007 mJ
Energy reduction FP32: 4.2×
Energy reduction INT8: 23.0×
Model loaded from ../pth/alexnet_10.pth
Sparsity: 10.0%
Energy: 0.139
Model loaded from ../pth/alexnet_30.pth
Sparsity: 30.0%
Energy: 0.108
Model loaded from ../pth/alexnet_50.pth
Sparsity: 50.0%
Energy: 0.077
Model loaded from ../pth/alexnet_70.pth
Sparsity: 70.0%
Energy: 0.046


In [14]:
from fvcore.nn import FlopCountAnalysis

model = get_model()
model.load_model(f"../pth/{mname}_fp32.pth", device='cpu')
x = torch.randn(1, 3, 32, 32)

flops = FlopCountAnalysis(model, x)
macs = flops.total() // 2  # FLOPs = 2 × MACs

print(f"MACs: {macs}")

Unsupported operator aten::max_pool2d encountered 3 time(s)
Unsupported operator aten::avg_pool2d encountered 1 time(s)


Model loaded from ../pth/squeezenet_fp32.pth
MACs: 26637312


In [15]:
def estimate_energy(macs, precision="fp32"):
    energy_per_mac = {
        "fp32": 4.6e-12,
        "fp16": 1.1e-12,
        "int8": 0.2e-12
    }
    return macs * energy_per_mac[precision]  # Joules

energy_fp32 = estimate_energy(macs, "fp32")
energy_fp16 = estimate_energy(macs, "fp16")
energy_int8 = estimate_energy(macs, "int8")

print(f"Energy FP32: {energy_fp32*1e3:.3f} mJ")
print(f"Energy FP16: {energy_fp16*1e3:.3f} mJ")
print(f"Energy INT8: {energy_int8*1e3:.3f} mJ")
print(f"Energy reduction FP32: {energy_fp32 / energy_fp16:.1f}×")
print(f"Energy reduction INT8: {energy_fp32 / energy_int8:.1f}×")

Energy FP32: 0.123 mJ
Energy FP16: 0.029 mJ
Energy INT8: 0.005 mJ
Energy reduction FP32: 4.2×
Energy reduction INT8: 23.0×


## Energy vs Sparsity

In [ ]:

mtitle = "ResNet"
resnet_results = [{
    "name": f"{mtitle} FP32",
    "accuracy": 0,
    "energy": 0.123,
    },
    {
        "name": f"{mtitle} FP16",
        "accuracy": 0,
        "energy": 0.029
    },
    {
        "name": f"{mtitle} INT8",
        "accuracy": 0,
        "energy": 0.005
    },
    {
        "name": f"{mtitle} FP32 Pruned 10%",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} FP32 Pruned 30%",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} FP32 Pruned 50%",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} FP32 Pruned 70%",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} Pruned and Quantized",
        "accuracy": 0,
        "energy": 0
    },
]

In [ ]:

mtitle = "SqueezeNet"
squeezenet_results = [{
    "name": f"{mtitle} FP32",
    "accuracy": 0,
    "energy": 0
    },
    {
        "name": f"{mtitle} FP16",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} INT8",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} FP32 Pruned 10%",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} FP32 Pruned 30%",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} FP32 Pruned 50%",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} FP32 Pruned 70%",
        "accuracy": 0,
        "energy": 0
    },
    {
        "name": f"{mtitle} Pruned and Quantized",
        "accuracy": 0,
        "energy": 0
    },
]

In [ ]:

mtitle = "AlexNet"

alexnet_results = [{
    "name": f"{mtitle} FP32",
    "accuracy": 0,
    "energy": 0.155
    },
    {
        "name": f"{mtitle} FP16",
        "accuracy": 0,
        "energy": 0.037
    },
    {
        "name": f"{mtitle} INT8",
        "accuracy": 0,
        "energy": 0.007
    },
    {
        "name": f"{mtitle} FP32 Pruned 10%",
        "accuracy": 87.43,
        "energy": 0.139
    },
    {
        "name": f"{mtitle} FP32 Pruned 30%",
        "accuracy": 88.55,
        "energy": 0.108
    },
    {
        "name": f"{mtitle} FP32 Pruned 50%",
        "accuracy": 88.44,
        "energy": 0.077
    },
    {
        "name": f"{mtitle} FP32 Pruned 70%",
        "accuracy": 88.94,
        "energy": 0.046
    },
    {
        "name": f"{mtitle} Pruned 70% and Quantized",
        "accuracy": 76.95,
        "energy": 0.002
    },
]